In [ ]:
from wildfires.analysis.plotting import *
from wildfires.data.datasets import *

In [ ]:
dataset = CCI_BurnedArea_MERIS_4_1()
observed = dataset.cubes.extract_strict(iris.Constraint("fraction of observed area"))

In [ ]:
cube_plotting(observed.collapsed("time", iris.analysis.MEAN))